In [24]:
!pip install langgraph==0.2.59
!pip install python-dotenv
!pip install --upgrade pip
!pip install -U langchain-openai
!pip install -U langchain_community

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached pydantic_settings-2.7.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached langchain_text_splitters-0.3.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.8 MB/s eta 0:00:00
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 41.4 MB/s eta 0:00:00
Using cached pydantic_settings-2.7.1-py3-none-any.whl (29 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.9 MB/s eta 0:00:00
Using cached langchain_text_splitters-0.3.5-py3-none-any.whl (31 kB)
Using cac

In [25]:
from dotenv import load_dotenv
load_dotenv()

False

In [26]:
#load_dotenv('/home/santhosh/Projects/courses/Pioneer/.env')
#load_dotenv('/Users/ssathishkumar/Desktop/analytics-vidya/LabMaterials/.env')

In [27]:
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode
from langchain_community.tools import TavilySearchResults
from IPython.display import Image

In [28]:
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [6]:
def should_continue(state: MessagesState) -> bool:
    messages = state['messages']
    last_message = messages[-1]
    if last_message.tool_calls:
        return 'tools'
    else:
        return END

In [7]:
def call_model(state: MessagesState):
    messages  = state['messages']
    response = llm.invoke(messages)
    
    return {'messages': messages + [response]}

In [8]:
graph = StateGraph(MessagesState)

In [ ]:
graph.add_node('agent', call_model)

In [10]:
search_tool = TavilySearchResults(max_results=5)

In [ ]:
tools = [search_tool]

tool_node = ToolNode(tools)

graph.add_node('tools', tool_node)

In [ ]:
# This means that the first node is the agent
# similar to setting graph.set_entry_point('agent')
graph.add_edge(START, "agent")

In [ ]:
graph.add_conditional_edges('agent', should_continue)

In [ ]:
graph.add_edge('tools', 'agent')

In [15]:
graph_compiled = graph.compile()

In [ ]:
Image(graph_compiled.get_graph().draw_mermaid_png())

In [ ]:
output = graph_compiled.invoke({'messages': [
    ('user', 'Write a simple report on how to use AI for productivity.')
]})

output

In [ ]:
from IPython.display import Markdown

Markdown(output['messages'][-1].content)